In [ ]:
import os
import tempfile

import matplotlib.pyplot as plt
import numpy as np
import omfiles as om
import xarray as xr

# Create a temporary directory for our demo files
temp_dir = tempfile.mkdtemp()
sample_file = os.path.join(temp_dir, "hierarchical_data.om")

## Create a sample hierarchical file

```
[Root Group] (unnamed)
│
├── temperature [24×2×2 float32 array]
│   ├── _ARRAY_DIMENSIONS = "HOUR,LOCATION,SENSOR" (scalar)
│   ├── units = "celsius" (scalar)
│   └── description = "Air temperature" (scalar)
│
├── humidity [24×2×2 float32 array]
│   ├── _ARRAY_DIMENSIONS = "HOUR,LOCATION,SENSOR" (scalar)
│   ├── units = "percent" (scalar)
│   └── description = "Relative humidity" (scalar)
│
├── pressure [24×2×2 float32 array]
│   ├── _ARRAY_DIMENSIONS = "HOUR,LOCATION,SENSOR" (scalar)
│   ├── units = "hPa" (scalar)
│   └── description = "Atmospheric pressure" (scalar)
│
├── random [10×10×10 float32 array]
│   ├── _ARRAY_DIMENSIONS = "X,Y,Z" (scalar)
│   ├── X [10 float32 array]
│   ├── Y [10 float32 array]
│   └── Z [10 float32 array]
│
├── HOUR [24 float32 array]
├── LOCATION [2 float32 array]
├── SENSOR [2 float32 array]
│
├── title = "Climate and random data sample" (scalar)
└── created = "2023-07-15" (scalar)
```

In [ ]:
# Create sample data with a hierarchical structure
def create_sample_file():
    """Create a sample OM file with hierarchical data structure"""
    writer = om.OmFilePyWriter(sample_file)

    # Create some sample data. The data always has the same shape of (24, 2, 2)
    temperatures = np.linspace(20, 30, 24 * 4).astype(np.float32).reshape((24, 2, 2))  # Temperature data
    humidity = np.random.uniform(40, 80, (24, 4)).astype(np.float32).reshape((24, 2, 2))  # Humidity data
    pressure = np.random.normal(1013, 5, (24, 4)).astype(np.float32).reshape((24, 2, 2))  # Pressure data

    # We save data with a chunk shape of (24,1,1) -> 24 hours for one location are compressed in separate chunks!
    chunk_shape = (24, 1, 1)

    # Create dimension arrays that will be used as coordinates
    hours = np.arange(24).astype(np.float32)
    locations = np.array([0, 1]).astype(np.float32)
    sensors = np.array([0, 1]).astype(np.float32)

    # Create dimension variables
    hours_var = writer.write_array(hours, name="HOUR", chunks=(24,))
    locations_var = writer.write_array(locations, name="LOCATION", chunks=(2,))
    sensors_var = writer.write_array(sensors, name="SENSOR", chunks=(2,))

    # Add dimension information and metadata as scalar variables
    temp_dims = writer.write_scalar("HOUR,LOCATION,SENSOR", name="_ARRAY_DIMENSIONS")
    temp_unit = writer.write_scalar("celsius", name="units")
    temp_desc = writer.write_scalar("Air temperature", name="description")

    humid_dims = writer.write_scalar("HOUR,LOCATION,SENSOR", name="_ARRAY_DIMENSIONS")
    humid_unit = writer.write_scalar("percent", name="units")
    humid_desc = writer.write_scalar("Relative humidity", name="description")

    press_dims = writer.write_scalar("HOUR,LOCATION,SENSOR", name="_ARRAY_DIMENSIONS")
    press_unit = writer.write_scalar("hPa", name="units")
    press_desc = writer.write_scalar("Atmospheric pressure", name="description")

    # Store data with hierarchical organization
    temp_var = writer.write_array(
        temperatures, chunks=chunk_shape, name="temperature", children=[temp_dims, temp_unit, temp_desc]
    )

    humid_var = writer.write_array(
        humidity, chunks=chunk_shape, name="humidity", children=[humid_dims, humid_unit, humid_desc]
    )

    press_var = writer.write_array(
        pressure, chunks=chunk_shape, name="pressure", children=[press_dims, press_unit, press_desc]
    )

    # Add some additional non-climate data with dimensions (10,10,10) chunked into (2,2,2)
    random_data = np.random.rand(10, 10, 10).astype(np.float32)
    random_dims = writer.write_scalar("X,Y,Z", name="_ARRAY_DIMENSIONS")
    x_coords = writer.write_array(np.arange(10).astype(np.float32), name="X", chunks=(10,))
    y_coords = writer.write_array(np.arange(10).astype(np.float32), name="Y", chunks=(10,))
    z_coords = writer.write_array(np.arange(10).astype(np.float32), name="Z", chunks=(10,))

    random_var = writer.write_array(
        random_data,
        chunks=(2, 2, 2),
        name="random",
        children=[
            random_dims,
            x_coords,
            y_coords,
            z_coords,
        ],  # dimension of random data is directly on random variable level
    )

    # Add global metadata
    global_attr = writer.write_scalar("Climate and random data sample", name="title")
    created_attr = writer.write_scalar("2023-07-15", name="created")

    # Add another group without a name which will serve as the entrypoint to the file
    root_group = writer.write_group(
        name="",
        children=[
            temp_var,
            humid_var,
            press_var,
            random_var,
            hours_var,
            locations_var,
            sensors_var,
            global_attr,
            created_attr,
        ],
    )

    writer.close(root_group)
    print(f"Created sample file at: {sample_file}")


# Create our sample file
create_sample_file()

In [ ]:
def open_and_analyze_with_xarray(file_path):
    """Open an OM file as an xarray Dataset and analyze its contents"""
    # Open the dataset using the OM engine
    ds = xr.open_dataset(file_path, engine="om")

    print("\n=== Dataset Information ===")
    print(f"Dataset dimensions: {dict(ds.dims)}")
    print(f"Dataset variables: {list(ds.variables)}")
    print(f"Dataset attributes: {ds.attrs}")

    # Display information about specific variables
    print("\n=== Climate Variables ===")
    for var_name in ["temperature", "humidity", "pressure"]:
        if var_name in ds:
            var = ds[var_name]
            print(f"\n{var_name}:")
            print(f"  - Dimensions: {var.dims}")
            print(f"  - Shape: {var.shape}")
            print(f"  - Attributes: {var.attrs}")
            print(f"  - Data type: {var.dtype}")
            print(
                f"  - Statistics: min={var.min().item():.2f}, max={var.max().item():.2f}, mean={var.mean().item():.2f}"
            )

    # Access and analyze temperature data
    if "temperature" in ds:
        temp = ds["temperature"]

        # Calculate daily average across all locations and sensors
        daily_avg = temp.mean(dim=["LOCATION", "SENSOR"])

        # Calculate location-specific averages
        location_avg = temp.mean(dim=["HOUR", "SENSOR"])

        # Example: Get data for a specific hour and location
        hour_6_loc_0 = temp.sel(HOUR=6, LOCATION=0)

        print("\n=== Temperature Analysis ===")
        print(f"Daily average temperature: \n{daily_avg.values}")
        print(f"Location average temperatures: \n{location_avg.values}")
        print(f"Temperature at hour 6, location 0: \n{hour_6_loc_0.values}")

    # Access the random data variable
    if "random" in ds:
        random_var = ds["random"]
        print("\n=== Random Data ===")
        print(f"Dimensions: {random_var.dims}")
        print(f"Shape: {random_var.shape}")

        # Calculate some statistics
        slice_stats = random_var.isel(Z=5)
        print(
            f"Z=5 slice statistics: min={slice_stats.min().item():.2f}, "
            f"max={slice_stats.max().item():.2f}, mean={slice_stats.mean().item():.2f}"
        )

    # Create some visualizations
    fig, axes = plt.subplots(2, 1, figsize=(12, 10))

    # Plot temperature time series for each location and sensor
    if "temperature" in ds:
        temp = ds["temperature"]
        for loc in range(len(ds.LOCATION)):
            for sen in range(len(ds.SENSOR)):
                temp.sel(LOCATION=loc, SENSOR=sen).plot(ax=axes[0], label=f"Loc {loc}, Sensor {sen}")

        axes[0].set_title("Temperature Time Series")
        axes[0].set_xlabel("Hour")
        axes[0].set_ylabel(f"Temperature ({temp.attrs.get('units', '')})")
        axes[0].legend()
        axes[0].grid(True)

    # Plot humidity vs pressure relationship
    if "humidity" in ds and "pressure" in ds:
        humid = ds["humidity"].mean(dim=["HOUR"]).values.flatten()
        press = ds["pressure"].mean(dim=["HOUR"]).values.flatten()

        axes[1].scatter(press, humid, alpha=0.7)
        axes[1].set_title("Humidity vs Pressure Relationship")
        axes[1].set_xlabel(f"Pressure ({ds['pressure'].attrs.get('units', '')})")
        axes[1].set_ylabel(f"Humidity ({ds['humidity'].attrs.get('units', '')})")
        axes[1].grid(True)

    plt.tight_layout()
    plt.show()

    return ds


# Open and analyze the dataset
dataset = open_and_analyze_with_xarray(sample_file)